In [1]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/hand-lotions-creams-bath-body-beauty/-/N-c5tkw?moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list = []

def scrape_products():
    global product_list, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Face Mask",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：87 条


In [3]:
df = pd.DataFrame(product_list)
df

product_name  \
0   Beloved Cherry Blossom & Tea Rose Hand Lotion,...   
1   Beloved Coconut & Warm Vanilla Hand Lotion, Tr...   
2   Baby Hand and Body Petroleum Jelly Jar - 7.5oz...   
3   Moisturizing Hand Cream - Coconut Milk & Shea ...   
4   Jergens Ultra Healing Hand and Body Lotion, Dr...   
..                                                ...   
82  Dionis Goat Milk Hand & Body Cream Milk & Hone...   
83  Dionis Goat Milk Hand & Body Cream Blood Orang...   
84  Skintensive Bruise Cream - Unscented Hand and ...   
85       Eden Body Works Coconut Hand Cream - 5 fl oz   
86  Dionis Goat Milk Hand & Body Cream Lavender Bl...   

                        brand   current_price           unit_price  \
0                     Beloved           $5.89        ($5.89/ounce)   
1                     Beloved           $5.89        ($5.89/ounce)   
2                       up&up           $3.29        ($0.44/ounce)   
3                       up&up           $3.39        ($1.13/ounce)   
4                     Jergens  $1.79 - $12.99                  N/A   
..                        ...             ...                  ...   
82  DIONIS GOAT MILK SKINCARE          $12.00                  N/A   
83  DIONIS GOAT MILK SKINCARE          $12.00                  N/A   
84                Skintensive          $29.94                  N/A   
85            Eden Body Works           $4.99  ($1.00/fluid ounce)   
86  DIONIS GOAT MILK SKINCARE          $12.00                  N/A   

                             star_rating review_counts  is_on_sale  \
0    4.5 out of 5 stars with 385 ratings   385 reviews           0   
1    4.4 out of 5 stars with 420 ratings   420 reviews           0   
2    4.7 out of 5 stars with 102 ratings   102 reviews           0   
3     4.4 out of 5 stars with 58 ratings    58 reviews           0   
4   4.7 out of 5 stars with 2770 ratings  2770 reviews           0   
..                                   ...           ...         ...   
82                                   N/A     0 reviews           0   
83       5 out of 5 stars with 1 ratings      1 review           0   
84    4.5 out of 5 stars with 15 ratings    15 reviews           0   
85    3.4 out of 5 stars with 14 ratings    14 reviews           0   
86                                   N/A     0 reviews           0   

   is_free_shipping  is_3day_delivery product_category  \
0                 0                 1        Face Mask   
1                 0                 1        Face Mask   
2                 0                 1        Face Mask   
3                 0                 1        Face Mask   
4               N/A                 1        Face Mask   
..              ...               ...              ...   
82                0                 0        Face Mask   
83                0                 0        Face Mask   
84                0                 0        Face Mask   
85                0                 1        Face Mask   
86                0                 0        Face Mask   

                                         product_link  
0   https://www.target.com/p/beloved-cherry-blosso...  
1   https://www.target.com/p/beloved-coconut-38-wa...  
2   https://www.target.com/p/baby-hand-and-body-pe...  
3   https://www.target.com/p/moisturizing-hand-cre...  
4   https://www.target.com/p/jergens-ultra-healing...  
..                                                ...  
82  https://www.target.com/p/dionis-goat-milk-hand...  
83  https://www.target.com/p/dionis-goat-milk-hand...  
84  https://www.target.com/p/skintensive-bruise-cr...  
85  https://www.target.com/p/eden-body-works-cocon...  
86  https://www.target.com/p/dionis-goat-milk-hand...  

[87 rows x 11 columns]

In [5]:
df.to_csv("handcream_data.csv",index=False)

# Page 4-6

In [6]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/hand-lotions-creams-bath-body-beauty/-/N-c5tkw?moveTo=product-list-grid&Nao=72"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list2 = []

def scrape_products():
    global product_list2, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list2.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Hand Cream",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list2)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
🔍 当前页面商品数: 29
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 3 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
🔁 成功加载到新页面
✅ WebDriver 关闭
📦 共抓取商品（去重后）：77 条


In [7]:
df2 = pd.DataFrame(product_list2)
df2

product_name  \
0   Cetaphil Soothing Hand and Body Cream with Alo...   
1   Native Limited Edition x Jarritos - Passion Fr...   
2   Origins Moisturizing Hand Cream - 2.53oz - Ult...   
3               Eczema Honey Oatmeal Hand Cream - 4oz   
4   Dionis Goat Milk Hand & Body Cream Vanilla Bea...   
..                                                ...   
72                     Shikai Mango Hand Cream - 2 fz   
73  Beloved White Gardenia & Coconut Water Hand Lo...   
74  Beloved Candied Lilac & Blackberry Hand Lotion...   
75  Beloved Mandarin & Cucumber Water Hand Lotion ...   
76  Jergens Hydrating Coconut Hand and Body Lotion...   

                        brand current_price           unit_price  \
0                    Cetaphil        $12.49        ($0.78/ounce)   
1                      Native        $16.99  ($1.03/fluid ounce)   
2                     Origins        $20.00                  N/A   
3                Eczema Honey        $23.49        ($5.87/ounce)   
4   DIONIS GOAT MILK SKINCARE        $12.00                  N/A   
..                        ...           ...                  ...   
72                     ShiKai        $27.49                  N/A   
73                    Beloved         $5.89        ($5.89/ounce)   
74                    Beloved         $5.99        ($5.99/ounce)   
75                    Beloved         $5.99        ($5.99/ounce)   
76                    Jergens         $9.59  ($0.57/fluid ounce)   

                            star_rating review_counts  is_on_sale  \
0   4.6 out of 5 stars with 194 ratings   194 reviews           0   
1   4.5 out of 5 stars with 185 ratings   185 reviews           0   
2     2.5 out of 5 stars with 4 ratings     4 reviews           0   
3   4.4 out of 5 stars with 187 ratings   187 reviews           0   
4       4 out of 5 stars with 2 ratings     2 reviews           0   
..                                  ...           ...         ...   
72                                  N/A     0 reviews           0   
73   4.5 out of 5 stars with 47 ratings    47 reviews           0   
74   4.7 out of 5 stars with 52 ratings    52 reviews           0   
75   4.7 out of 5 stars with 44 ratings    44 reviews           0   
76  4.6 out of 5 stars with 361 ratings   361 reviews           0   

   is_free_shipping  is_3day_delivery product_category  \
0                 0                 1       Hand Cream   
1                 0                 1       Hand Cream   
2                 0                 1       Hand Cream   
3                 0                 1       Hand Cream   
4                 0                 0       Hand Cream   
..              ...               ...              ...   
72                0                 0       Hand Cream   
73                0                 1       Hand Cream   
74                0                 1       Hand Cream   
75                0                 1       Hand Cream   
76                0                 1       Hand Cream   

                                         product_link  
0   https://www.target.com/p/cetaphil-soothing-gel...  
1   https://www.target.com/p/native-limited-editio...  
2   https://www.target.com/p/origins-moisturizing-...  
3   https://www.target.com/p/eczema-honey-oatmeal-...  
4   https://www.target.com/p/dionis-goat-milk-hand...  
..                                                ...  
72  https://www.target.com/p/shikai-mango-hand-cre...  
73  https://www.target.com/p/beloved-white-gardeni...  
74  https://www.target.com/p/beloved-candied-lilac...  
75  https://www.target.com/p/beloved-mandarin-38-c...  
76  https://www.target.com/p/jergens-hyrdating-coc...  

[77 rows x 11 columns]

In [8]:
df2.to_csv("handcream_data2.csv",index=False)

# 汇总前6页

In [9]:
df_combined_handcream_1_6 = pd.concat([df,df2],axis=0, ignore_index=True)
df_combined_handcream_1_6

product_name    brand  \
0    Beloved Cherry Blossom & Tea Rose Hand Lotion,...  Beloved   
1    Beloved Coconut & Warm Vanilla Hand Lotion, Tr...  Beloved   
2    Baby Hand and Body Petroleum Jelly Jar - 7.5oz...    up&up   
3    Moisturizing Hand Cream - Coconut Milk & Shea ...    up&up   
4    Jergens Ultra Healing Hand and Body Lotion, Dr...  Jergens   
..                                                 ...      ...   
159                     Shikai Mango Hand Cream - 2 fz   ShiKai   
160  Beloved White Gardenia & Coconut Water Hand Lo...  Beloved   
161  Beloved Candied Lilac & Blackberry Hand Lotion...  Beloved   
162  Beloved Mandarin & Cucumber Water Hand Lotion ...  Beloved   
163  Jergens Hydrating Coconut Hand and Body Lotion...  Jergens   

      current_price           unit_price  \
0             $5.89        ($5.89/ounce)   
1             $5.89        ($5.89/ounce)   
2             $3.29        ($0.44/ounce)   
3             $3.39        ($1.13/ounce)   
4    $1.79 - $12.99                  N/A   
..              ...                  ...   
159          $27.49                  N/A   
160           $5.89        ($5.89/ounce)   
161           $5.99        ($5.99/ounce)   
162           $5.99        ($5.99/ounce)   
163           $9.59  ($0.57/fluid ounce)   

                              star_rating review_counts  is_on_sale  \
0     4.5 out of 5 stars with 385 ratings   385 reviews           0   
1     4.4 out of 5 stars with 420 ratings   420 reviews           0   
2     4.7 out of 5 stars with 102 ratings   102 reviews           0   
3      4.4 out of 5 stars with 58 ratings    58 reviews           0   
4    4.7 out of 5 stars with 2770 ratings  2770 reviews           0   
..                                    ...           ...         ...   
159                                   N/A     0 reviews           0   
160    4.5 out of 5 stars with 47 ratings    47 reviews           0   
161    4.7 out of 5 stars with 52 ratings    52 reviews           0   
162    4.7 out of 5 stars with 44 ratings    44 reviews           0   
163   4.6 out of 5 stars with 361 ratings   361 reviews           0   

    is_free_shipping  is_3day_delivery product_category  \
0                  0                 1        Face Mask   
1                  0                 1        Face Mask   
2                  0                 1        Face Mask   
3                  0                 1        Face Mask   
4                N/A                 1        Face Mask   
..               ...               ...              ...   
159                0                 0       Hand Cream   
160                0                 1       Hand Cream   
161                0                 1       Hand Cream   
162                0                 1       Hand Cream   
163                0                 1       Hand Cream   

                                          product_link  
0    https://www.target.com/p/beloved-cherry-blosso...  
1    https://www.target.com/p/beloved-coconut-38-wa...  
2    https://www.target.com/p/baby-hand-and-body-pe...  
3    https://www.target.com/p/moisturizing-hand-cre...  
4    https://www.target.com/p/jergens-ultra-healing...  
..                                                 ...  
159  https://www.target.com/p/shikai-mango-hand-cre...  
160  https://www.target.com/p/beloved-white-gardeni...  
161  https://www.target.com/p/beloved-candied-lilac...  
162  https://www.target.com/p/beloved-mandarin-38-c...  
163  https://www.target.com/p/jergens-hyrdating-coc...  

[164 rows x 11 columns]

In [10]:
df_combined_handcream_1_6.to_csv("handcream_1_6.csv",index=False)

# Page 7-9

In [2]:
import time
import random
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

# ========== 1. User-Agent 列表（仅桌面端 Chrome） ==========
# 🚀 伪装 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_6_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Fedora) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6420.5 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_7_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6410.50 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; ARM64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Arch) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6400.98 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; ZorinOS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6390.75 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; Elementary OS) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36",
    "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64; MX Linux) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6370.20 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko Chrome/120.0.6350.90 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6350.90 Safari/537.36"
]




random_user_agent = random.choice(USER_AGENTS)

# ========== 2. 初始化 WebDriver ==========
service = Service("/Users/zhuxiaowen/Desktop/chromedriver-mac-arm64/chromedriver")
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument(f"user-agent={random_user_agent}")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=service, options=options)
print(f"✅ 启动 WebDriver, 使用UA: {random_user_agent}")

# ========== 3. 访问页面 ==========
base_url = "https://www.target.com/c/hand-lotions-creams-bath-body-beauty/-/N-c5tkw?Nao=144&moveTo=product-list-grid"
driver.get(base_url)
time.sleep(random.uniform(5, 10))

# ========== 4. 修改点①：等待商品卡片加载完成 ==========
def wait_until_products_loaded(min_count=25, timeout=15):
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: len(d.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")) >= min_count
        )
    except TimeoutException:
        print(f"⚠️ 超时未加载出 {min_count} 个商品，当前页可能不完整")

# ========== 5. 修改点②：强化滚动逻辑，直到商品数量不再增加 ==========
def scroll_and_wait():
    last_height = driver.execute_script("return document.body.scrollHeight")
    prev_count = 0

    for i in range(30):
        print(f"🔄 滚动页面: 第 {i+1} 次")
        driver.execute_script("window.scrollBy(0, 1200);")
        time.sleep(random.uniform(2, 4))

        products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
        if len(products) > prev_count:
            prev_count = len(products)
        else:
            print("✅ 页面懒加载完成")
            break

        try:
            if products:
                ActionChains(driver).move_to_element(products[-1]).perform()
        except:
            pass

    time.sleep(3)

# ========== 🔧 修改点①：记录每一页的首个商品链接 ==========
visited_first_links = set()

product_list3 = []

def scrape_products():
    global product_list3, visited_first_links
    scroll_and_wait()
    wait_until_products_loaded()

    products = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-4fd1fd45-0')]")
    print(f"🔍 当前页面商品数: {len(products)}")

    if not products:
        print("⚠️ 本页无商品，跳过")
        return

    # 🔧 修改点②：判断是否回到了已访问页面（用首个商品链接判重）
    try:
        first_link = products[0].find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
    except:
        first_link = None

    if first_link in visited_first_links:
        print("🔁 检测到页面内容重复，疑似回到了第一页，停止抓取")
        raise StopIteration
    visited_first_links.add(first_link)

    for product in products:
        try:
            title_element = product.find_element(By.XPATH, ".//a[@data-test='product-title']//div")
            name = title_element.text.strip()
        except NoSuchElementException:
            continue

        try:
            brand = product.find_element(By.XPATH, ".//a[@data-test='@web/ProductCard/ProductCardBrandAndRibbonMessage/brand']").text.strip()
        except:
            brand = "N/A"

        try:
            price = product.find_element(By.XPATH, ".//span[@data-test='current-price']").text.strip()
        except:
            price = "N/A"

        try:
            unit_price = product.find_element(By.XPATH, ".//span[@data-test='unit-price']").text.strip()
        except:
            unit_price = "N/A"

        try:
            rating = product.find_element(By.XPATH, ".//span[@data-test='ratings']").text.strip()
        except:
            rating = "N/A"

        try:
            reviews = product.find_element(By.XPATH, ".//span[@data-test='rating-count']").text.strip()
        except:
            reviews = "0 reviews"

        try:
            product.find_element(By.XPATH, ".//div[@data-test='comparison-price']")
            is_on_sale = 1
        except:
            is_on_sale = 0

        try:
            if " - " in price:
                is_free_shipping = "N/A"
            else:
                price_value = float(price.replace("$", ""))
                is_free_shipping = 1 if price_value >= 35 else 0
        except:
            is_free_shipping = "N/A"

        try:
            shipping_info = product.find_element(By.XPATH, ".//div[@data-test='LPFulfillmentSectionShippingFA_standardShippingMessage']").text
            if "get it tomorrow" in shipping_info:
                is_3day_delivery = 1
            else:
                match = re.search(r"arrives\s+\w+,\s+([A-Za-z]+ \d+)", shipping_info)
                if match:
                    delivery_str = match.group(1)
                    delivery_date = datetime.datetime.strptime(f"{delivery_str} {datetime.datetime.now().year}", "%b %d %Y").date()
                    days_diff = (delivery_date - datetime.datetime.now().date()).days
                    is_3day_delivery = 1 if 0 <= days_diff <= 3 else 0
                else:
                    is_3day_delivery = "N/A"
        except:
            is_3day_delivery = "N/A"

        try:
            link = product.find_element(By.XPATH, ".//a[@data-test='product-title']").get_attribute("href")
        except:
            link = "N/A"

        product_list3.append({
            "product_name": name,
            "brand": brand,
            "current_price": price,
            "unit_price": unit_price,
            "star_rating": rating,
            "review_counts": reviews,
            "is_on_sale": is_on_sale,
            "is_free_shipping": is_free_shipping,
            "is_3day_delivery": is_3day_delivery,
            "product_category": "Hand Cream",
            "product_link": link
        })

# 翻页逻辑（原样）
def go_to_next_page(prev_first_link):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='next']"))
        )
        driver.execute_script("arguments[0].click();", next_button)
        print("✅ 点击下一页")
    except:
        print("❌ 没有找到下一页按钮，结束爬取")
        return False

    # 等待新页面加载且首个商品链接发生变化
    for _ in range(15):
        time.sleep(random.uniform(1.5, 2.5))
        try:
            products = driver.find_elements(By.XPATH, "//a[@data-test='product-title']")
            if products:
                new_first_link = products[0].get_attribute("href")
                if new_first_link != prev_first_link:
                    print("🔁 成功加载到新页面")
                    return True
        except:
            pass
    print("⚠️ 页面跳转失败或未加载新内容")
    return False


# ========== 主循环（加异常捕捉 StopIteration） ==========
max_pages = 3
try:
    for page_num in range(max_pages):
        print(f"\n📄 正在爬取第 {page_num + 1} 页...")
        scrape_products()

        # 获取当前页面首个链接作为翻页后的参照
        current_first_link = next(iter(visited_first_links), None)
        if not go_to_next_page(current_first_link):
            break
except StopIteration:
    print("⛔ 检测到重复页，主动中止爬虫")


driver.quit()
print("✅ WebDriver 关闭")

print(f"📦 共抓取商品（去重后）：{len(product_list3)} 条")

✅ 启动 WebDriver, 使用UA: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6381.60 Safari/537.36

📄 正在爬取第 1 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
🔁 成功加载到新页面

📄 正在爬取第 2 页...
🔄 滚动页面: 第 1 次
🔄 滚动页面: 第 2 次
🔄 滚动页面: 第 3 次
✅ 页面懒加载完成
⚠️ 超时未加载出 25 个商品，当前页可能不完整
🔍 当前页面商品数: 24
✅ 点击下一页
⚠️ 页面跳转失败或未加载新内容
✅ WebDriver 关闭
📦 共抓取商品（去重后）：48 条


In [3]:
df3 = pd.DataFrame(product_list3)
df3

product_name  \
0   Beloved White Gardenia & Coconut Water Hand Lo...   
1   Beloved Candied Lilac & Blackberry Hand Lotion...   
2   Beloved Mandarin & Cucumber Water Hand Lotion ...   
3   Jergens Hydrating Coconut Hand and Body Lotion...   
4   Vaseline Intensive Care Cocoa Radiant Hand and...   
5   DR. SQUATCH Hand Lotion - Wood Barrel Bourbon ...   
6    DR. SQUATCH Hand Lotion - Fresh Falls - 10 fl oz   
7       DR. SQUATCH Hand Lotion - Pine Tar - 10 fl oz   
8     Origins Ginger Hand & Body Lotion - Ulta Beauty   
9   Molton Brown Orange & Bergamot Hand Lotion, 10 oz   
10  Molton Brown Heavenly Gingerlily Hand Lotion 1...   
11  Molton Brown Coastal Cypress & Sea Fennel Hand...   
12  Native Limited Edition x Jarritos - Passion Fr...   
13  Native Limited Edition x Jarritos - Watermelon...   
14  Molton Brown Coastal Cypress & Sea Fennel Hand...   
15  CND Hand & Body - Scentsations - Mango & Cocon...   
16  Aveeno Stress Relief Moisturizing Body Lotion ...   
17             Corn Huskers Hand Treatment Lotion 7Oz   
18  CND Hand & Body - Scentsations - Black Cherry ...   
19  Diptyque Emulsion Velours Velvet Hand Lotion, ...   
20  Hempz Beauty Actives Herbal Hand And Body Loti...   
21  Triple Lanolin Original Formula Hand & Body Lo...   
22  Bio-Oil Hydrating Hand and Body Lotion Unscent...   
23  All Good Goop on the Go Hand and Body Lotion -...   
24  Molton Brown Delicious Rhubarb & Rose Hand Lot...   
25  Avalon Organics Hand and Body Lotion Lavender ...   
26  CND Scentsations Hand & Body Lotion | Quick-ab...   
27  Uncle Bud's Agave Hand and Body Lotions Agave ...   
28  Image Skincare Vital C Hydrating Hand & Body L...   
29                    EltaMD So Silky Hand Creme 3 oz   
30   Molton Brown Fiery Pink Pepper Hand Lotion 10 oz   
31   L'Occitane Shea Butter Intensive Hand Balm, 5 oz   
32    Oribe Cote d'Azur Nourishing Hand Creme, 3.4 oz   
33               L'Occitane Almond Hand and Nail Care   
34            Diptyque Exfoliating Hand Wash, 11.8 oz   
35  Alba Botanica Hawaiian Hand and Body Lotion Co...   
36  Kiss My Face Lavender and Ylang Ylang Hand & B...   
37  Avalon Organics Hand and Body Lotion Lemon 12 ...   
38  Kiss My Face Rosemary and Tea Tee Goat Milk Ha...   
39  Kiss My Face Kids Fragrance Free Hand and Body...   
40  Triple Lanolin Mango Vera Hand And Body Lotion...   
41  Shikai Gardenia Moisturizing Hand and Body Lot...   
42  Shikai Yuzu Moisturizing Hand and Body Lotion ...   
43  Shikai Sandalwood Moisturizing Hand and Body L...   
44  Shikai Coconut Moisturizing Hand and Body Loti...   
45                  Aveda Shampure Hand and Body Wash   
46  Shikai Cucumber Melon Moisturizing Hand and Bo...   
47  Evideco French Home Goods Elegant Soap Dispens...   

                        brand    current_price           unit_price  \
0                     Beloved            $5.89        ($5.89/ounce)   
1                     Beloved            $5.99        ($5.99/ounce)   
2                     Beloved            $5.99        ($5.99/ounce)   
3                     Jergens            $9.59  ($0.57/fluid ounce)   
4                    Vaseline            $2.49        ($1.25/ounce)   
5                 DR. SQUATCH           $19.59  ($1.96/fluid ounce)   
6                 DR. SQUATCH           $19.59  ($1.96/fluid ounce)   
7                 DR. SQUATCH           $19.59  ($1.96/fluid ounce)   
8                     Origins  $20.00 - $29.00                  N/A   
9                Molton Brown           $31.45                  N/A   
10               Molton Brown           $34.99                  N/A   
11               Molton Brown           $31.45                  N/A   
12                     Native           $16.99  ($1.03/fluid ounce)   
13                     Native           $16.99  ($1.03/fluid ounce)   
14               Molton Brown           $34.99                  N/A   
15                        CND           $13.99                  N/A   
16                     Aveeno  $11.19 - 

In [4]:
df3.to_csv("handcream_7_9.csv",index=False)